# keras 基本RNN
keras.layers.SimpleRNN() 值得注意的是return_sequences参数 True:各时间步的ht都输出 False仅输出最后时间步的ht
输入维度[batch, seq_len, feature]

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.datasets import load_iris
import os
import pandas as pd
import numpy as np
import tensorflow.keras as keras
from keras import Model
from tensorflow.keras import Model
from sklearn.preprocessing import MinMaxScaler
from PIL import Image
os.environ['CUDA_VISIBLE_DEVICES'] = '1' # 使用 GPU 1
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0],True)
logical_devices = tf.config.list_logical_devices("GPU")

# 简单实验 用one-hot矩阵进行RNN实验

In [4]:
input_word = 'abcde'
char2id = {'a':0, 'b':1, 'c':2, 'd':3, 'e':4}
id2one_hot = {0 : [1., 0., 0., 0., 0.],
              1:[0., 1., 0., 0., 0.],
              2:[0., 0., 1., 0., 0.],
              3:[0., 0., 0., 1., 0.],
              4:[0., 0., 0., 0., 1.]}
# one-hot编码 abcde

In [21]:
x_train = [id2one_hot[char2id['a']], id2one_hot[char2id['b']], id2one_hot[char2id['c']], id2one_hot[char2id['d']], id2one_hot[char2id['e']]]
y_train = [char2id['b'], char2id['c'], char2id['d'], char2id['e'], char2id['a']]

In [22]:
np.random.seed(50)
np.random.shuffle(x_train)
np.random.seed(50)
np.random.shuffle(y_train)
tf.random.set_seed(50)

In [26]:
x_train = np.array(x_train).reshape((len(x_train), 1, 5))
y_train = np.array(y_train).reshape(len(y_train), 1)
print(x_train.shape)
print(y_train.shape)

(5, 1, 5)
(5, 1)


In [27]:
model = keras.models.Sequential([
    keras.layers.SimpleRNN(units=8),
    keras.layers.Dense(5, activation='softmax')
])

In [28]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['sparse_categorical_accuracy']
)

In [29]:
checkpoint_save_path = 'tensorflow_study/chapter6/model_dir/rnn_one_hot_model.ckpt'

cp_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_save_path,
    save_weights_only=True,
    save_best_only=True,
    monitor='loss',
)

In [30]:
history = model.fit(x_train, y_train, batch_size=2, epochs=100, callbacks=[cp_callback])

Epoch 1/100
3/3 [==============================] - 2s 49ms/step - loss: 1.5813 - sparse_categorical_accuracy: 0.2000
Epoch 2/100
3/3 [==============================] - 0s 12ms/step - loss: 1.5724 - sparse_categorical_accuracy: 0.2000
Epoch 3/100
3/3 [==============================] - 0s 12ms/step - loss: 1.5649 - sparse_categorical_accuracy: 0.2000
Epoch 4/100
3/3 [==============================] - 0s 12ms/step - loss: 1.5583 - sparse_categorical_accuracy: 0.2000
Epoch 5/100
3/3 [==============================] - 0s 12ms/step - loss: 1.5519 - sparse_categorical_accuracy: 0.2000
Epoch 6/100
3/3 [==============================] - 0s 12ms/step - loss: 1.5449 - sparse_categorical_accuracy: 0.2000
Epoch 7/100
3/3 [==============================] - 0s 12ms/step - loss: 1.5383 - sparse_categorical_accuracy: 0.2000
Epoch 8/100
3/3 [==============================] - 0s 13ms/step - loss: 1.5313 - sparse_categorical_accuracy: 0.2000
Epoch 9/100
3/3 [==============================] - 0s 15ms/step 

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 8)                 112       
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
Total params: 157
Trainable params: 157
Non-trainable params: 0
_________________________________________________________________


In [34]:
test_input = [id2one_hot[char2id['a']], id2one_hot[char2id['e']],id2one_hot[char2id['d']],id2one_hot[char2id['a']]]
test_input = np.array(test_input).reshape((1,4,5))
test_input.shape

(1, 4, 5)

In [44]:
char_id = tf.math.argmax(model.predict(test_input), axis=1)
char_id.numpy()

array([1])

# Embedding
keras.layers.Embedding(词汇表大小, embedding维度)
注意：输入是[batch, seq_len]的int矩阵！ int值的取值范围是[0,词汇表大小) 输出是[batch, seq_len, embedding]

In [52]:
x_train = [[char2id['a'], char2id['b'], char2id['c'], char2id['e'], char2id['e']],
           [char2id['d'], char2id['d'], char2id['c'], char2id['a'], char2id['a']],
           [char2id['e'], char2id['e'], char2id['c'], char2id['b'], char2id['e']],
           [char2id['a'], char2id['b'], char2id['c'], char2id['d'], char2id['c']],
           [char2id['b'], char2id['c'], char2id['c'], char2id['d'], char2id['a']],]
y_train = [char2id['a'], char2id['b'], char2id['e'], char2id['d'], char2id['c']]

In [53]:
np.random.seed(50)
np.random.shuffle(x_train)
np.random.seed(50)
np.random.shuffle(y_train)

In [54]:
x_train = np.array(x_train)
y_train = np.array(y_train).reshape((len(y_train),1))
print(x_train.shape)
print(y_train.shape)

(5, 5)
(5, 1)


In [59]:
x_train

array([[0, 1, 2, 3, 2],
       [4, 4, 2, 1, 4],
       [3, 3, 2, 0, 0],
       [1, 2, 2, 3, 0],
       [0, 1, 2, 4, 4]])

In [ ]:
model_with_embedding = keras.models.Sequential([
    keras.layers.Embedding(input_dim=5, output_dim=3),
    keras.layers.SimpleRNN(units=8),
    keras.layers.Dense(units=5, activation='softmax')
])

In [56]:
model_with_embedding.compile(
    optimizer=keras.optimizers.Adam(),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['sparse_categorical_accuracy']
)

In [57]:
checkpoint_save_path_embedding = 'tensorflow_study/chapter6/model_dir/rnn_embedding_model.ckpt'

cp_callback_embedding = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_save_path_embedding,
    save_weights_only=True,
    save_best_only=True,
    monitor='loss',
)

In [64]:
history_embedding = model_with_embedding.fit(x_train, y_train, batch_size=2, epochs=100, callbacks=[cp_callback_embedding])

Epoch 1/100
3/3 [==============================] - 1s 20ms/step - loss: 1.6150 - sparse_categorical_accuracy: 0.2000
Epoch 2/100
3/3 [==============================] - 0s 17ms/step - loss: 1.6043 - sparse_categorical_accuracy: 0.4000
Epoch 3/100
3/3 [==============================] - 0s 17ms/step - loss: 1.5970 - sparse_categorical_accuracy: 0.4000
Epoch 4/100
3/3 [==============================] - 0s 17ms/step - loss: 1.5898 - sparse_categorical_accuracy: 0.4000
Epoch 5/100
3/3 [==============================] - 0s 17ms/step - loss: 1.5830 - sparse_categorical_accuracy: 0.4000
Epoch 6/100
3/3 [==============================] - 0s 16ms/step - loss: 1.5766 - sparse_categorical_accuracy: 0.4000
Epoch 7/100
3/3 [==============================] - 0s 16ms/step - loss: 1.5673 - sparse_categorical_accuracy: 0.4000
Epoch 8/100
3/3 [==============================] - 0s 15ms/step - loss: 1.5600 - sparse_categorical_accuracy: 0.4000
Epoch 9/100
3/3 [==============================] - 0s 17ms/step 